# Découper les données de BRATS avec SimpleITK

## Import des librairies

In [2]:
import glob

!pip install tqdm
from tqdm import tqdm 

import numpy as np

!pip install SimpleITK
import SimpleITK as sitk

## Chargement des fichiers à découper

In [34]:
threshold=0.01 #treshold du nombre de segmentation dans une image
flair_all=glob.glob("./BRATS/*_flair.nii.gz")
flair_all.sort()

seg_all=glob.glob("./BRATS/*_seg.nii.gz")
seg_all.sort()
nb_display=3
print( f" -> First {nb_display} train items ")
[print(flair_all[i], seg_all[i]) for i in range(nb_display)]

print( f" -> Last {nb_display} train items ")
[print(flair_all[i], seg_all[i]) for i in range(len(flair_all)-nb_display,len(flair_all))]

 -> First 3 train items 
./BRATS/BraTS20_Training_001_flair.nii.gz ./BRATS/BraTS20_Training_001_seg.nii.gz
./BRATS/BraTS20_Training_002_flair.nii.gz ./BRATS/BraTS20_Training_002_seg.nii.gz
./BRATS/BraTS20_Training_003_flair.nii.gz ./BRATS/BraTS20_Training_003_seg.nii.gz
 -> Last 3 train items 
./BRATS/BraTS20_Training_367_flair.nii.gz ./BRATS/BraTS20_Training_367_seg.nii.gz
./BRATS/BraTS20_Training_368_flair.nii.gz ./BRATS/BraTS20_Training_368_seg.nii.gz
./BRATS/BraTS20_Training_369_flair.nii.gz ./BRATS/BraTS20_Training_369_seg.nii.gz


[None, None, None]

## Découper les données

In [30]:
for nImage in tqdm(range(len(flair_all))):
    Zs=[]
    flair = sitk.ReadImage(flair_all[nImage])
    seg = sitk.ReadImage(seg_all[nImage])
    seg =  sitk.Cast(seg,sitk.sitkUInt8)
    size = list(flair.GetSize())
    totalPixel=size[1]*size[0]
    tresholdValue=totalPixel*threshold
    stat=sitk.StatisticsImageFilter()
    for z in range(0,size[2],5):
         stat.Execute(seg[:,:,z])
         Zs.append([stat.GetSum(),z])
    Zs.sort(reverse=True) 
    size[2]=0
    
    for index in range(min(5,len(Zs))):
        if Zs[index][0]<tresholdValue:
            break
        Extractor = sitk.ExtractImageFilter() 
        indexExtrac = [0, 0, Zs[index][1]]
        Extractor.SetSize(size)
        Extractor.SetIndex(indexExtrac)
        flair_slice=Extractor.Execute(flair)
        flair_slice=sitk.RescaleIntensity(flair_slice,0,255)
        flair_slice = sitk.Cast(flair_slice,sitk.sitkUInt8)
        total=0
        
        if nImage+1<=300:
            sitk.WriteImage(flair_slice, "Test/Train/"+f"{(nImage+1):03d}"+"_"+str(index+1)+"flair.png",imageIO='PNGImageIO')
            sitk.WriteImage(Extractor.Execute(seg), "Test/Train/"+f"{(nImage+1):03d}"+"_"+str(index+1)+"seg.png",imageIO='PNGImageIO')
        else:
            sitk.WriteImage(flair_slice, "Test/Test1/"+f"{(nImage+1):03d}"+"_"+str(index+1)+"flair.png",imageIO='PNGImageIO')
            sitk.WriteImage(Extractor.Execute(seg), "Test/Test1/"+f"{(nImage+1):03d}"+"_"+str(index+1)+"seg.png",imageIO='PNGImageIO')

100%|██████████| 369/369 [00:47<00:00,  7.80it/s]
